In [1]:
#IMPORTING LIBRARIES
import numpy as np 
import pandas as pd
import cv2
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import zipfile

In [3]:
#IMPORTING DATASET
dataset = zipfile.ZipFile("/content/Untitled Folder/kagglecatsanddogs_5340.zip")
dataset.extractall()
img = []
label = []
for i in os.listdir("PetImages"):
  for j in os.listdir("PetImages/"+i):
    if i ==  "Cat":
      label.append(0)
    else:
      label.append(1)
    img.append(os.path.join("PetImages",i,j)) 


In [ ]:
#CREATING DATAFRAME
img_df = pd.DataFrame(img)
img_label = pd.DataFrame(label)
df = pd.concat([img_df,img_label],axis=1)
df.columns = ["Image","Label"]



In [ ]:
#DELETING THE FILES WHICH IS NOT AN IMAGE
for i in df["Image"]:
  if  i[-3:] != "jpg":
    df.drop(axis=0)

In [ ]:
# VISUALIZING WEIGHT
import seaborn as sns
sns.countplot(df["label"])

In [ ]:
#DROPPING THE WRONG DATA 
df = df[df["Image"] != 'PetImages\\Cat\\666.jpg']
df = df[df["Image"] != 'PetImages\\Cat\\Thumbs.db']
df = df[df["Image"] != 'PetImages\\Dog\\11702.jpg']
df = df[df["Image"] != 'PetImages\\Dog\\Thumbs.db']
df = df[df["Image"] != 'PetImages\\Cat\\10493.jpg']

In [ ]:

tf.image.resize(
    df["Image"],
    [500,500],
    method=ResizeMethod.BILINEAR,
    preserve_aspect_ratio=False,
    antialias=False,
    name=None
)

In [ ]:
#DATA VISUALIZATION

from tensorflow.keras.preprocessing.image import load_img
dogs = []

dogs = df["Image"].loc[df["Label"] == 1]
for i in range(25):
    asd = random.randint(12501,25000)
    dogs.append(df["Image"].iloc[asd])
for index,i in enumerate(list):
    plt.subplot(5,5,index+1) 
    image = load_img(i)
    plt.imshow(image)
    plt.title("Dogs")
    plt.axis("off")
    
import random
cats = []
cats = df["Image"].loc[df["Label"] == 0]
for i in range(25):
    asd = random.randint(0,12499)
    cats.append(df["Image"].iloc[asd])

for index,i in enumerate(list_2):
    plt.subplot(5,5,index+1) 
    image = load_img(i)
    plt.imshow(image)
    plt.title("Cats")
    plt.axis("off")

In [ ]:
#MODELIZATION
from sklearn.model_selection import train_test_split

df['Label'] = df['Label'].astype('str')
train,test = train_test_split(df,test_size=0.33,random_state=5)


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator(
    rescale = 1./255,  
    rotation_range = 40, 
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

val_generator = ImageDataGenerator(rescale = 1./255)

train_iterator = train_generator.flow_from_dataframe(
    train, 
    x_col='Image', 
    y_col='Label', 
    target_size=(128,128), 
    batch_size=512, 
    class_mode='binary'
)

val_iterator = val_generator.flow_from_dataframe(
    test, 
    x_col='Image', 
    y_col='Label', 
    target_size=(128,128), 
    batch_size=512, 
    class_mode='binary'
)
test_images = val_generator.flow_from_dataframe(
    dataframe=df,
    x_col='Image',
    y_col='Label',
    target_size=(128, 128),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=False)

In [ ]:
from keras import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
model = Sequential([
                    Conv2D(16, (3,3), activation='relu', input_shape=(128,128,3)),
                    MaxPool2D((2,2)),
                    Conv2D(32, (3,3), activation='relu'),
                    MaxPool2D((2,2)),
                    Conv2D(64, (3,3), activation='relu'),
                    MaxPool2D((2,2)),
                    Flatten(),
                    Dense(512, activation='relu'),
                    Dense(1, activation='sigmoid')
])

In [ ]:
#TRAIN AND SCORE SCORE
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
print("-----------------------------------------------------------------------")
history = model.fit(train_iterator, epochs=10, validation_data=val_iterator)

results = model.evaluate(test_images, verbose=0)

print("Loss -> {:.2f}".format(results[0]))
print("Accuracy Point -> {:.2f}%".format(results[1] * 100))

In [ ]:
#HEATMAP
from sklearn.metrics import confusion_matrix, classification_report
pred = (model.predict(test_images) >= 0.5).astype(np.int)

cm = confusion_matrix(test_images.labels, pred, labels=[0, 1])
clr = classification_report(test_images.labels, pred, labels=[0, 1])
                        
LABELS=["Cat", "Dog"]

plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='g', vmin=0, cmap='Blues', cbar=False)
plt.xticks(ticks=[0.5, 1.5], labels=["Cat", "Dog"])
plt.yticks(ticks=[0.5, 1.5], labels=["Cat", "Dog"])
plt.xlabel("Prediction")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
score = history.history['accuracy']
val_score = history.history['val_accuracy']
epochs = range(len(score))

plt.plot(epochs, score, 'b', label='Training Accuracy')
plt.plot(epochs, val_score, 'r', label='Validation Accuracy')
plt.title('Accuracy Graph')
plt.legend()
plt.figure()

loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(epochs, loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Loss Graph')
plt.legend()
plt.show()